In [ ]:
pip install --upgrade pip

In [ ]:
pip install pytrends

In [ ]:
pip install pandas

In [ ]:
pip install matplotlib

In [ ]:
pip install wordcloud

In [ ]:
pip install seaborn

In [ ]:
from pytrends.request import TrendReq
import pandas as pd
from IPython.display import display, HTML

# Create pytrends object
pytrends = TrendReq()

# Read keyword input from the user
keyword_input = input("Enter a keyword value: ")

# Build payload to get related queries for the main keyword
try:
    pytrends.build_payload(kw_list=[keyword_input], geo='US', timeframe='today 12-m')
    related_queries_main = pytrends.related_queries().values()
except Exception as e:
    print("Error occurred while fetching related queries:", e)
    related_queries_main = []

# Extract queries from the dictionary values for the main keyword
queries = []
for query_data in related_queries_main:
    if query_data['top'] is not None:
        queries.extend(query_data['top']['query'].tolist())

# Define the number of results per batch
results_per_batch = 25

# List to store all related queries
related_queries_all = []

# Process queries in batches
for query in queries:
    try:
        pytrends.build_payload(kw_list=[query], geo='US', timeframe='today 12-m')
        related_queries = pytrends.related_queries().values()
        for query_data in related_queries:
            if query_data['top'] is not None:
                query_list = query_data['top']['query'].tolist()
                value_list = query_data['top']['value'].tolist()
                for related_query, value in zip(query_list, value_list):
                    # Check if the query already exists in related_queries_all
                    query_exists = False
                    for existing_query in related_queries_all:
                        if existing_query['query'] == related_query:
                            existing_query['value'] += value
                            query_exists = True
                            break
                    if not query_exists:
                        related_queries_all.append({'query': related_query, 'value': value})
    except Exception as e:
        print("Error occurred while processing query:", query, e)

# Sort the related queries by value in descending order
related_queries_all_sorted = sorted(related_queries_all, key=lambda x: x['value'], reverse=True)

# Convert the sorted related queries data into a pandas DataFrame
df = pd.DataFrame(related_queries_all_sorted)

# Display the DataFrame as a scrollable table
display(HTML(df.to_html()))

# Export data to CSV with user-specified filename
csv_filename = f"{keyword_input}.csv"
df.to_csv(csv_filename, index=False)
print(f"Data exported to {csv_filename}")


In [ ]:
from pytrends.request import TrendReq
import pandas as pd
import matplotlib.pyplot as plt

# Create pytrends object
pytrends = TrendReq()

# Read keyword input from the user
keyword = input("Enter a keyword value: ")

# Build payload to get search interest by region
pytrends.build_payload(kw_list=[keyword], geo='US', timeframe='today 12-m')

# Get search interest by region
state_data = pytrends.interest_by_region(resolution='REGION', inc_low_vol=True, inc_geo_code=False)

# Convert state codes to state names
state_names = []
for state_code in state_data.index:
    try:
        state_name = pytrends.suggestions(state_code)[0]['title']
    except IndexError:
        state_name = state_code  # Use state code if state name not found
    state_names.append(state_name)
state_data['State'] = state_names

# Sort state-wise search interest data by values in descending order
state_data_sorted = state_data.sort_values(by=keyword, ascending=False)

# Plot state-wise keyword search interest
plt.figure(figsize=(12, 6))
ax = state_data_sorted.plot(kind='bar', figsize=(15, 8), color='skyblue')
plt.title(f'State-wise Search Interest for the Keyword "{keyword}" in the US (Past Year)', fontsize=16)
plt.xlabel('State', fontsize=14)
plt.ylabel('Search Interest', fontsize=14)
plt.xticks(rotation=45, ha='right', fontsize=12)  # Rotate x-axis labels for readability
plt.yticks(fontsize=12)
plt.grid(axis='y', linestyle='--', alpha=0.7)  # Add grid lines for reference

# Add values on top of each bar
for bar in ax.patches:
    ax.annotate(str(int(bar.get_height())), (bar.get_x() + bar.get_width() / 2, bar.get_height()), ha='center', va='bottom', fontsize=10)

plt.tight_layout()  # Adjust layout to prevent clipping of labels
plt.show()

# Export data to CSV with user-specified filename
csv_filename = f"{keyword}_search_interest.csv"
state_data_sorted.to_csv(csv_filename)
print(f"State-wise search interest data exported to {csv_filename}")


In [ ]:
import matplotlib.pyplot as plt

# Convert the sorted related queries data into a pandas DataFrame
related_queries_df = pd.DataFrame(related_queries_all_sorted)

# Display the top 15 related queries
top_15_queries = related_queries_df.head(15)

# Pie Chart
plt.figure(figsize=(10, 8))
plt.pie(top_15_queries['value'], labels=top_15_queries['query'], autopct='%1.1f%%', startangle=140)
plt.title('Top 15 Related Queries Distribution')
plt.axis('equal')
plt.show()


In [ ]:
related_queries_dataframe = pd.DataFrame(related_queries_all_sorted)
related_queries_dataframe.shape[0]

In [ ]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud

# Convert the sorted related queries data into a pandas DataFrame
related_queries_dataframe = pd.DataFrame(related_queries_all_sorted)

# Generate the word cloud
wordcloud = WordCloud(
    width=1920,  # HD width
    height=1080,  # HD height
    background_color='white',
    colormap='viridis',  # color map for the word cloud
    contour_color='steelblue',  # contour color
    contour_width=2,  # contour width
    max_words=related_queries_dataframe.shape[0],  # maximum number of words to display
    prefer_horizontal=0.8,  # prefer horizontal orientation of words
    min_font_size=10,  # minimum font size
    max_font_size=200,  # maximum font size for HD clarity
    random_state=42  # random state for reproducibility
).generate(' '.join(related_queries_dataframe['query']))

# Plot the word cloud
plt.figure(figsize=(19.20, 10.80))  # HD aspect ratio
plt.imshow(wordcloud, interpolation='bilinear')
plt.title('Related Queries Word Cloud', fontsize=24)  # Increased title font size
plt.axis('off')
plt.tight_layout()  # Adjust layout for better appearance
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Sort the DataFrame by the "value" column in descending order
sorted_df = df.sort_values(by='value', ascending=False)

# Set plot style to use high-definition output
sns.set_context("talk")

# Violin Plot
plt.figure(figsize=(14, 8))
sns.violinplot(data=sorted_df, y='value', color='skyblue', linewidth=2, inner='quartile')
plt.title('Violin Plot of Search Interest (Sorted by Value)', fontsize=20)
plt.xlabel('Search Interest', fontsize=16)
plt.ylabel('Distribution', fontsize=16)
plt.grid(True, linestyle='--', alpha=0.7)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
# plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Set a consistent color palette
palette = sns.color_palette("viridis", n_colors=len(df))
plt.figure(figsize=(1920/100, 1080/100), dpi=300)  # 1920x1080 resolution

# Scatter Plot
plt.figure(figsize=(12, 8))
sns.scatterplot(data=df, x=df.index, y='value', hue='query', palette=palette, s=100, alpha=0.7)
plt.title('Scatter Plot of Search Interest')
plt.xlabel('Query Index')
plt.ylabel('Search Interest')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xticks(rotation=45)

# Adjust margins
plt.subplots_adjust(bottom=0.1, top=0.9, left=0.1, right=0.9)

plt.show()
